> ### Matrix_QM_parallel.ipynb
> ### Parallel Lanczos Algorithm for 1D Quantum Mechanics
> ### Hybrid MPI + OpenMP implementation with domain decomposition
> ### Author: Dhruv Patel (2130292), Mohammadreza Khansari (2132180)
> ### Date: 31st March 2025

In [1]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4458265 sha256=1dd0efe32364c1799064bae4bf20696b8b2bf02dc584fb7ce40eceaa50aad3b2
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [2]:
import numpy as np
from mpi4py import MPI
from scipy.sparse.linalg import eigsh

# =============================================================================
# Lanczos Algorithm Implementation (Parallel Version)
# =============================================================================

def Lanczos_algorithm(m, n, a, rank, num_procs, comm):
    """
    Implements the parallel Lanczos algorithm using MPI domain decomposition

    Each MPI process works on a local segment of the full wavefunction
    Communication is required to exchange boundary values between processes

    Args:
        m (int): Krylov subspace size (iteration)
        n (int): Total number of grid points
        a (float): Grid spacing
        rank (int): MPI rank of the current process
        num_procs (int): Total number of MPI processes
        comm (MPI.Comm): MPI communicator

    Returns:
        eigenvalues (ndarray): 10 smallest eigenvalues (root process only)
    """

    # Domain decomposition: Splitting grid points across processes
    sizes = distribute_elements(n, num_procs)
    local_n = sizes[rank]
    starts = [sum(sizes[:i]) for i in range(num_procs)]
    start_idx = starts[rank]

    # Precomputing the local position array for current process
    M = (n - 1) // 2
    x_local = a * np.arange(start_idx - M, start_idx - M + local_n)

    # Initializing Lanczos vectors and coefficients
    v = np.zeros((local_n, m))    # Orthonormal basis
    B = np.zeros(m-1)             # Off-diagonal elements
    alpha = np.zeros(m)           # Diagonal elements

    # Initializing the starting vector (local part) and normalizing it globally
    v1 = np.random.randn(local_n)
    v1 = normalize_vector(v1, comm)

    # First Lanczos iteration: applying Ĥ using boundary exchange
    v_temp = exchange_boundary_data(v1, local_n, rank, num_procs, comm)
    w1_p = Hv(v_temp, a, x_local, comm)  # Now returns local_n elements
    a1 = dot_product(w1_p, v1, comm)
    w1 = w1_p - a1 * v1
    alpha[0] = a1
    v[:, 0] = v1

    comm.Barrier()
    start_time = MPI.Wtime()  # Start timing after initialization

    # Lanczos iteration loop
    for j in range(1, m):
        # Computing the beta coefficient with global norm
        B[j - 1] = get_global_norm(w1, comm)

        # Handling the Lanczos breakdown
        if np.isclose(B[j-1], 0.0):
            # Restart if numerical breakdown occurs
            v[:, j] = normalize_vector(np.random.randn(local_n), comm)
        else:
            v[:, j] = w1 / B[j-1]

        # Function call: Modified Gram-Schmidt reorthogonalization
        v = modified_gram_schmidt(v, j + 1, comm)

        # Check orthogonality after every 10 steps
        if j % 10 == 0:
            check_orthogonality(v, j + 1, comm)

        # Applying the Hamiltonian operator (with boundary communication)
        v_temp = exchange_boundary_data(v[:, j], local_n, rank, num_procs, comm)
        w1_p = Hv(v_temp, a, x_local, comm)
        alpha[j] = dot_product(w1_p, v[:, j], comm)
        w1 = w1_p - alpha[j]*v[:, j] - B[j - 1]*v[:, j - 1]

    comm.Barrier()
    iteration_time = MPI.Wtime() - start_time

    # Root process (rank 0) solves tridiagonal system and evaluate eigenvalues
    if rank == 0:
        T = np.diag(alpha) + np.diag(B, -1) + np.diag(B, 1)
        eigenvalues, _ = eigsh(T, k=10, which='SM')
        print(f"Iteration time: {iteration_time:.4f}s (excluding tridiagonal solve)")
    else:
        eigenvalues = None

    return eigenvalues

# =============================================================================
# MPI Helper Functions
# =============================================================================

def distribute_elements(N, num_procs):
    """
    Distributes grid points evenly across MPI processes

    Returns:
        sizes (list): Number of points per process
    """
    base, rem = divmod(N, num_procs)
    return [base + 1 if i < rem else base for i in range(num_procs)]

def exchange_boundary_data(v, local_n, rank, size, comm):
    """
    Performs non-blocking communication to exchange ghost cell data with neighboring processes

    This is needed because the Hamiltonian operator (finite differences) accesses neighboring elements

    Args:
        v (ndarray): Local vector segment
        local_n (int): Local dimension
        rank (int): Current process rank
        size (int): Total number of processes
        comm (MPI.Comm): MPI communicator

    Returns:
        ndarray: Vector with ghost cells from neighbors
    """
    right_rank = (rank + 1) % size
    left_rank = (rank - 1) % size

    # Preparing the send/receive buffers
    send_buf = np.array([v[0], v[-1]], dtype=np.float64)
    recv_buf = np.empty(2, dtype=np.float64)

    # Non-blocking communications
    reqs = [
        comm.Isend(send_buf[0:1], dest=left_rank),
        comm.Isend(send_buf[1:2], dest=right_rank),
        comm.Irecv(recv_buf[0:1], source=right_rank),
        comm.Irecv(recv_buf[1:2], source=left_rank)
    ]
    MPI.Request.Waitall(reqs)

    # Building the extended local vector with ghost cells at boundaries
    v_ext = np.empty(local_n + 2, dtype=np.float64)
    v_ext[1:-1] = v         # Local data
    v_ext[0] = recv_buf[1]  # From left neighbor's right
    v_ext[-1] = recv_buf[0] # From right neighbor's left
    return v_ext

def check_orthogonality(v, k, comm):
    """Check orthogonality of vectors up to index `k`

    Args:
        v (ndarray): Matrix of vectors
        k (int): Index up to which to check orthogonality
        comm (MPI.Comm): MPI communicator

    Returns:
        None
    """
    ortho_errors = []
    for i in range(k):
        for j in range(i, k):

            # Compute global dot product (parallel)
            dot = dot_product(v[:, i], v[:, j], comm)
            if i == j:
                error = abs(dot - 1.0)  # Should be 1
            else:
                error = abs(dot)        # Should be 0
            ortho_errors.append((i, j, error))

    # Print max error from rank 0
    if comm.rank == 0:
        max_error = max([e for (_, _, e) in ortho_errors])
        print(f"Max orthogonality error after step {k}: {max_error:.2e}")
        if max_error > 1e-10:
            print("WARNING: Orthogonality lost!")

def normalize_vector(v, comm):
    """Normalizes the vector using a global norm computed via MPI"""
    return v / get_global_norm(v, comm)

def get_global_norm(v, comm):
    """Computes the L2 norm of a vector across all MPI processes"""
    local_sq = np.sum(v**2)
    global_sq = comm.allreduce(local_sq, op=MPI.SUM)
    return np.sqrt(global_sq)

def dot_product(v, u, comm):
    """Computes the global dot product of two vectors across MPI processes"""
    local_dot = v @ u
    return comm.allreduce(local_dot, op=MPI.SUM)

def Hv(v, a, x_local, comm):
    """Vectorized Hamiltonian application"""
    """
    Applies the Vectorized Hamiltonian operator to a local segment of the
    wavefunction to both kinetic and potential parts

    Args:
        v (ndarray): Local segment of ψ with ghost cells
        a (float): Grid spacing
        x_local (ndarray): Local position array
        comm (MPI.Comm): MPI communicator

    Returns:
        result (ndarray): Local segment of Hψ
    """
    hbar = 1.0  # Assuming natural units

    # Kinetic energy (sparse momentum operator)
    kinetic = (hbar**2 / (2*a**2)) * (2*v[1:-1] - v[:-2] - v[2:])

    # Potential energy
    potential = 0.5 * np.abs(x_local)**2 * v[1:-1]
    return kinetic + potential  # Returns local_n elements

def modified_gram_schmidt(matrix, k, comm):
    """
    Applies the Parallel modified Modified Gram-Schmidt orthogonalization
    Reorthogonalizes the current Lanczos vectors for better numerical stability

    Args:
        matrix (ndarray): Column vectors to orthogonalize
        k (int): Number of vectors to process
        comm (MPI.Comm): MPI communicator

    Returns:
        ndarray: Orthonormalized vectors
    """
    for i in range(k):
        # Normalizing the current vector
        orthogonalized = matrix.copy()
        norm = get_global_norm(orthogonalized[:, i], comm)
        orthogonalized[:, i] /= norm

        # Orthogonalizing the subsequent vectors
        for j in range(i + 1, k):
            proj = dot_product(orthogonalized[:, j], orthogonalized[:, i], comm)
            orthogonalized[:, j] -= proj * orthogonalized[:, i]
    return orthogonalized

# =============================================================================
# Main Execution
# =============================================================================

if __name__ == "__main__":

    # MPI initialization
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    # Physical parameters
    L = 25.0    # Physical system size
    N = 2049    # Number of spatial grid points (2^16): ensuring 'odd' for symmetry
    m = 90      # Krylov subspace size: controls Lanczos iterations
    a = L / N   # Grid spacing

    # Synchronize before timing
    comm.Barrier()
    start_time = MPI.Wtime()

    # Executing the algorithm to evaluate eigenvalues
    eigenvalues = Lanczos_algorithm(m, N, a, rank, size, comm)

    end_time = MPI.Wtime()

    # Saving the results (root process only)
    if rank == 0:

        # Storing the computed eigenvalues in text file
        print("\n-----------------------------------------\n")
        print(f"Computed eigenvalues: {eigenvalues}")
        exec_time = end_time - start_time
        print(f"\nExecution time: {exec_time:.4f} seconds")
        np.savetxt('eigenvalues_parallel.txt', eigenvalues,
                   header=f"Eigenvalues computed with N = {N}, L = {L}, a = {a:.4f} (parallel execution)\nExecution time: {exec_time:.4f} seconds")

        # Storing the computed Relative error in text file
        # Expected eigenvalues: E_n = ħω(n + 1/2)
        # For our parameters ω = sqrt(k/m) = 1 (natural units)
        expected = [0.5 + i for i in range(10)]
        relative_error = (eigenvalues - expected) / expected
        print(f"\nRelative error:", relative_error)
        np.savetxt('relative_error_parallel.txt', relative_error,
                   header="Relative error computed from eigenvalues and expected values (parallel execution)")

Max orthogonality error after step 11: 1.94e-15
Max orthogonality error after step 21: 3.62e-15
Max orthogonality error after step 31: 4.58e-15
Max orthogonality error after step 41: 6.36e-15
Max orthogonality error after step 51: 9.31e-15
Max orthogonality error after step 61: 1.27e-14
Max orthogonality error after step 71: 1.50e-14
Max orthogonality error after step 81: 1.77e-14
Iteration time: 6.4108s (excluding tridiagonal solve)

-----------------------------------------

Computed eigenvalues: [  3.52792068  14.00078376  31.74586727  56.57273612  83.56219275
 126.33345481 177.60271357 232.9663294  298.62896338 361.52874642]

Execution time: 6.4167 seconds

Relative error: [ 6.05584137  8.33385584 11.69834691 15.16363889 17.56937617 21.96971906
 26.3234944  30.06217725 34.13281922 37.05565752]


In [3]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
print(cores)

96


Default Nodes and CPUs per Runtime Type

------------------------------

1. ***CPU Runtime:***

Nodes: 1 (single VM instance).

CPUs: Typically 2 physical cores (4 logical threads via hyper-threading). Use os.cpu_count() to check (e.g., returns 2 for physical cores or 4 for logical threads).

------------------------------
2. ***T4 GPU Runtime:***

Nodes: 1 (same VM as CPU runtime).

CPUs: Same as CPU runtime (2 physical cores).

GPU: 1 NVIDIA T4 GPU.

------------------------------
3. ***v2-8 TPU Runtime:***

Nodes: 1 (single TPU node).

CPUs: Same as CPU runtime (2 physical cores on the VM).

TPU Cores: 8 TPU cores (part of the TPU accelerator, not CPU cores).

------------------------------